### Connect libraries

In [32]:
import petl as etl
from freshdesk.api import API
from datetime import date

### Connect to Freshdesk

In [4]:
a = API('yourname.freshdesk.com', 'gj34pg98hyatoken')


            Freshdesk has deprecated their V1 API from 1st July, 2018.
            For more info, visit https://support.freshdesk.com/support/solutions/articles/231955-important-deprecation-of-api-v1
            
            For more info about freshdesk V2 API, visit https://developers.freshdesk.com/api/
            
            Now python-freshdesk library will by default return V2 API client. You need to migrate your project accordingly.
            
            
        


### Get tickets

In [64]:
tickets = a.tickets.list_tickets(updated_since='2015-01-01', filter_name=None)

### Put into an array

In [78]:
tickets_array = []
for t in tickets:   
    tickets_array.append({'id': t.id,
                          'subject':t.subject,
                          #'company_id': t.company_id,
                          'created_at': t.created_at.date(),
                          'due_by': t.due_by, # в отличие от created_at это просто строка. Ну и так сойдет.
                          'fr_due_by': t.fr_due_by, # Снова строка, ну ок
                          'priority': t.priority,
                          'requester_id': t.requester_id,
                          'responder_id': t.responder_id,
                          'status': t.status,
                          'type': t.type,
                          'source': a.tickets.get_ticket(t.id)._source,
                          'tags': ' '.join(t.tags),
                          'description_text': a.tickets.get_ticket(t.id).description_text,                                                 
                         })

### Make a table from the array

In [79]:
table1 = etl.fromdicts(tickets_array, header=['id','subject', 'due_by', 'fr_due_by',
                                             'priority', 'requester_id', 'responder_id', 'status',
                                             'type', 'source', 'tags', 'description_text', 'created_at' ])

### Trasform the table

###### *Снова ограничение: python библиотека отдает не коды статусов тикета, а сразу расшифровки. Причем неправильно расшифровывает кастомные статусы. Так что нужно забирать напрямую у Freshdesk. Пока просто неправильные кастомные статусы будут переименованы вручную, их немного.*

In [80]:
table2 = etl.convert(table1, 'status', {'status_8': 'В ожидании третьей стороны',
                                        'status_9': 'Тестирование',
                                        'status_10': 'В разработке',
                                        'status_11': 'Запланирована удаленная сессия',
                                       })

### Export to Excel

In [81]:
etl.toxlsx(table2, 'tickets.xlsx')